# Ciphertext‑Only AES CPA (Last Round) + Master Key Recovery
This notebook uses **ciphertexts + power traces only** (no plaintext) to recover the **Round 10 key** via CPA and then inverts the AES-128 key schedule to obtain the **original master key**.

In [36]:

# Imports and utilities
import numpy as np
import re
from typing import Tuple


In [37]:

# AES inverse S-Box (for last-round CPA) and Hamming weight lookup
INV_SBOX = np.array([
    0x52,0x09,0x6A,0xD5,0x30,0x36,0xA5,0x38,0xBF,0x40,0xA3,0x9E,0x81,0xF3,0xD7,0xFB,
    0x7C,0xE3,0x39,0x82,0x9B,0x2F,0xFF,0x87,0x34,0x8E,0x43,0x44,0xC4,0xDE,0xE9,0xCB,
    0x54,0x7B,0x94,0x32,0xA6,0xC2,0x23,0x3D,0xEE,0x4C,0x95,0x0B,0x42,0xFA,0xC3,0x4E,
    0x08,0x2E,0xA1,0x66,0x28,0xD9,0x24,0xB2,0x76,0x5B,0xA2,0x49,0x6D,0x8B,0xD1,0x25,
    0x72,0xF8,0xF6,0x64,0x86,0x68,0x98,0x16,0xD4,0xA4,0x5C,0xCC,0x5D,0x65,0xB6,0x92,
    0x6C,0x70,0x48,0x50,0xFD,0xED,0xB9,0xDA,0x5E,0x15,0x46,0x57,0xA7,0x8D,0x9D,0x84,
    0x90,0xD8,0xAB,0x00,0x8C,0xBC,0xD3,0x0A,0xF7,0xE4,0x58,0x05,0xB8,0xB3,0x45,0x06,
    0xD0,0x2C,0x1E,0x8F,0xCA,0x3F,0x0F,0x02,0xC1,0xAF,0xBD,0x03,0x01,0x13,0x8A,0x6B,
    0x3A,0x91,0x11,0x41,0x4F,0x67,0xDC,0xEA,0x97,0xF2,0xCF,0xCE,0xF0,0xB4,0xE6,0x73,
    0x96,0xAC,0x74,0x22,0xE7,0xAD,0x35,0x85,0xE2,0xF9,0x37,0xE8,0x1C,0x75,0xDF,0x6E,
    0x47,0xF1,0x1A,0x71,0x1D,0x29,0xC5,0x89,0x6F,0xB7,0x62,0x0E,0xAA,0x18,0xBE,0x1B,
    0xFC,0x56,0x3E,0x4B,0xC6,0xD2,0x79,0x20,0x9A,0xDB,0xC0,0xFE,0x78,0xCD,0x5A,0xF4,
    0x1F,0xDD,0xA8,0x33,0x88,0x07,0xC7,0x31,0xB1,0x12,0x10,0x59,0x27,0x80,0xEC,0x5F,
    0x60,0x51,0x7F,0xA9,0x19,0xB5,0x4A,0x0D,0x2D,0xE5,0x7A,0x9F,0x93,0xC9,0x9C,0xEF,
    0xA0,0xE0,0x3B,0x4D,0xAE,0x2A,0xF5,0xB0,0xC8,0xEB,0xBB,0x3C,0x83,0x53,0x99,0x61,
    0x17,0x2B,0x04,0x7E,0xBA,0x77,0xD6,0x26,0xE1,0x69,0x14,0x63,0x55,0x21,0x0C,0x7D
], dtype=np.uint8)

# Hamming weight for 0..255
HW = np.unpackbits(np.arange(256, dtype=np.uint8)[:, None], axis=1).sum(axis=1).astype(np.int16)


In [38]:
# Load ciphertexts (32-char hex string format)
import numpy as np

with open("cipher_texts.txt", "r") as f:
    lines = [ln.strip() for ln in f if ln.strip()]

cprs = []
for s in lines:
    b = bytes.fromhex(s)
    if len(b) != 16:
        raise ValueError(f"Ciphertext not 16 bytes: {s}")
    cprs.append(np.frombuffer(b, dtype=np.uint8))

cprs = np.vstack(cprs)
print("Loaded", len(cprs), "ciphertexts")

Loaded 5000 ciphertexts


In [39]:
# Load traces from a ZIP containing .txt files (space-separated integers)
import zipfile

trace_zip_path = "power_traces.zip"
traces = []

with zipfile.ZipFile(trace_zip_path, "r") as zf:
    txt_files = sorted([f for f in zf.namelist() if f.endswith(".txt")])
    for fname in txt_files:
        with zf.open(fname) as f:
            raw = f.read().decode("utf-8").strip()
            samples = [float(x) for x in raw.split()]
            traces.append(samples)

# Convert to numpy array (pad if needed)
max_len = max(len(t) for t in traces)
traces = np.array([t + [0.0]*(max_len-len(t)) if len(t)<max_len else t[:max_len] for t in traces])

print("Loaded", traces.shape[0], "traces of length", traces.shape[1])

# Keep traces and ciphertexts aligned
n = min(len(cprs), len(traces))
cprs = cprs[:n]
traces = traces[:n]
print("Using", n, "ciphertext-trace pairs")

Loaded 5000 traces of length 12
Using 5000 ciphertext-trace pairs


In [40]:

# If your original notebook already defines `traces`, you can skip this cell.
# Otherwise, provide your numpy array of shape (N, T) in `traces`.
# Example placeholder (disabled):
# import numpy as np
# traces = np.load('traces.npy')  # shape (N, T)
# assert traces.shape[0] == cprs.shape[0]


In [41]:

# CPA on AES last round using ciphertexts + traces
def center_columns(X: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    means = X.mean(axis=0, dtype=np.float64)
    Xc = X - means
    stds = Xc.std(axis=0, dtype=np.float64)
    stds[stds == 0] = 1.0
    return Xc, stds

def cpa_last_round(cprs: np.ndarray, traces: np.ndarray):
    N, T = traces.shape
    traces = traces.astype(np.float64)
    traces_c, traces_std = center_columns(traces)
    denom_traces = traces_std * np.sqrt(N)

    best_key = np.zeros(16, dtype=np.uint8)
    per_byte = []

    for b in range(16):
        cbyte = cprs[:, b]
        best_corr = -1.0
        best_k = 0
        best_t = 0

        for k in range(256):
            inter = INV_SBOX[np.bitwise_xor(cbyte, k)]
            h = HW[inter].astype(np.float64)
            h -= h.mean()
            denom_h = np.sqrt((h*h).sum())
            if denom_h == 0: 
                continue
            numer = h @ traces_c
            corr = np.abs(numer / (denom_h * denom_traces))
            t_idx = int(np.argmax(corr))
            cval = float(corr[t_idx])
            if cval > best_corr:
                best_corr, best_k, best_t = cval, k, t_idx

        best_key[b] = best_k
        per_byte.append((b, best_k, best_corr, best_t))
        print(f"[Byte {b:2d}] k=0x{best_k:02x}  max|r|={best_corr:.6f}  at t={best_t}")

    return best_key, per_byte


In [42]:

# Invert AES-128 key schedule: round 10 key -> original master key
SBOX = np.array([
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
], dtype=np.uint8)

RCON = [0x00,0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36]

def invert_key_schedule(last_round_key: np.ndarray) -> np.ndarray:
    rk = last_round_key.astype(np.uint8).copy().tolist()
    for r in range(10, 0, -1):
        prev = rk[:]  # copy
        # XOR back columns 3->0
        for i in range(12, 16):
            prev[i-12] ^= prev[i]
        # inverse key schedule core on first word
        a0,a1,a2,a3 = prev[12],prev[13],prev[14],prev[15]
        a0,a1,a2,a3 = SBOX[a1],SBOX[a2],SBOX[a3],SBOX[a0]  # inv RotWord + SubWord
        a0 ^= RCON[r]
        prev[0] ^= a0; prev[1] ^= a1; prev[2] ^= a2; prev[3] ^= a3
        rk = prev[:16]
    return np.frombuffer(bytes(rk[:16]), dtype=np.uint8)


In [43]:

# Runner: expects `traces` to be available (shape N x T)
# If `traces` isn't defined, raise a clear error.
try:
    traces
except NameError:
    raise NameError("Variable `traces` not found. Please load your traces into a numpy array named `traces` (shape N x T).")

if traces.shape[0] != cprs.shape[0]:
    n = min(traces.shape[0], cprs.shape[0])
    print(f"Warning: counts differ (ciphertexts {cprs.shape[0]} vs traces {traces.shape[0]}). Truncating to {n}.")
    traces = traces[:n]
    cprs = cprs[:n]

last_round_key, per_byte = cpa_last_round(cprs, np.asarray(traces))
print("\nRecovered Round 10 key:", ''.join(f'{b:02x}' for b in last_round_key))

orig_key = invert_key_schedule(last_round_key)
print("Original AES-128 key:", ''.join(f'{b:02x}' for b in orig_key))


[Byte  0] k=0xd0  max|r|=0.251335  at t=10
[Byte  1] k=0x14  max|r|=0.241488  at t=10
[Byte  2] k=0xf9  max|r|=0.247232  at t=10
[Byte  3] k=0xa8  max|r|=0.254190  at t=10
[Byte  4] k=0xc9  max|r|=0.254150  at t=10
[Byte  5] k=0xee  max|r|=0.247097  at t=10
[Byte  6] k=0x25  max|r|=0.246304  at t=10
[Byte  7] k=0x89  max|r|=0.269895  at t=10
[Byte  8] k=0xe1  max|r|=0.253748  at t=10
[Byte  9] k=0x3f  max|r|=0.278795  at t=10
[Byte 10] k=0x0c  max|r|=0.233311  at t=10
[Byte 11] k=0xc8  max|r|=0.241821  at t=10
[Byte 12] k=0xb6  max|r|=0.257587  at t=10
[Byte 13] k=0x63  max|r|=0.270538  at t=10
[Byte 14] k=0x0c  max|r|=0.234670  at t=10
[Byte 15] k=0xa6  max|r|=0.277245  at t=10

Recovered Round 10 key: d014f9a8c9ee2589e13f0cc8b6630ca6
Original AES-128 key: 0214f9a8c9ee2589e13f0cc8b6630ca6


In [44]:
# Display recovered keys
print("\nRecovered Round 10 key:")
print(' '.join(f'{b:02x}' for b in last_round_key))

orig_key = invert_key_schedule(last_round_key)
print("\nRecovered Original AES-128 Key:")
print(' '.join(f'{b:02x}' for b in orig_key))



Recovered Round 10 key:
d0 14 f9 a8 c9 ee 25 89 e1 3f 0c c8 b6 63 0c a6

Recovered Original AES-128 Key:
02 14 f9 a8 c9 ee 25 89 e1 3f 0c c8 b6 63 0c a6


In [45]:
# Pure Python verification of recovered key (no Crypto needed)

# AES S-box
sbox = [
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]

Rcon = [0x00,0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36]

def sub_word(word):
    return ((sbox[(word >> 24) & 0xFF] << 24) |
            (sbox[(word >> 16) & 0xFF] << 16) |
            (sbox[(word >> 8) & 0xFF] << 8) |
             sbox[word & 0xFF])

def rot_word(word):
    return ((word << 8) & 0xFFFFFFFF) | ((word >> 24) & 0xFF)

def key_expansion(key_bytes):
    w = [0]*44
    for i in range(4):
        w[i] = (key_bytes[4*i]<<24)|(key_bytes[4*i+1]<<16)|(key_bytes[4*i+2]<<8)|key_bytes[4*i+3]
    for i in range(4,44):
        temp = w[i-1]
        if i % 4 == 0:
            temp = sub_word(rot_word(temp)) ^ (Rcon[i//4] << 24)
        w[i] = w[i-4] ^ temp
    round_keys = []
    for i in range(11):
        rk = []
        for j in range(4):
            word = w[4*i+j]


In [46]:

# Correct AES-128 key schedule inversion from Round 10 key -> Master key
# Replaces the previous invert_key_schedule implementation

SBOX = np.array([
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
], dtype=np.uint8)

RCON = [0x00,0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36]

def bytes_to_words(b):
    # 16 bytes -> 4 big-endian 32-bit words
    return [ (b[4*i]<<24) | (b[4*i+1]<<16) | (b[4*i+2]<<8) | b[4*i+3] for i in range(4) ]

def words_to_bytes(ws):
    out = []
    for w in ws:
        out.extend([(w>>24)&0xFF,(w>>16)&0xFF,(w>>8)&0xFF,w&0xFF])
    return bytes(out)

def sub_word(w):
    return (int(SBOX[(w>>24)&0xFF])<<24) | (int(SBOX[(w>>16)&0xFF])<<16) | (int(SBOX[(w>>8)&0xFF])<<8) | int(SBOX[w&0xFF])

def rot_word(w):
    return ((w<<8)&0xFFFFFFFF) | ((w>>24)&0xFF)

def invert_key_schedule_from_r10(r10_bytes):
    # r10_bytes = bytes of round-10 key (16B), arranged as standard AES rk[10]
    w = [0]*44
    w[40:44] = bytes_to_words(r10_bytes)
    # Invert words down to w[0..3]
    for i in range(43, 3, -1):
        if i % 4 == 0:
            # w[i] = w[i-4] ^ SubWord(RotWord(w[i-1])) ^ (Rcon[i/4] << 24)
            # => w[i-4] = w[i] ^ SubWord(RotWord(w[i-1])) ^ (Rcon[i/4] << 24)
            temp = sub_word(rot_word(w[i-1])) ^ (RCON[i//4] << 24)
            w[i-4] = w[i] ^ temp
        else:
            # w[i] = w[i-4] ^ w[i-1]  =>  w[i-4] = w[i] ^ w[i-1]
            w[i-4] = w[i] ^ w[i-1]
    return words_to_bytes(w[0:4])

def expand_key_forward(master_key_bytes):
    # standard expansion to all round keys (11 * 16 bytes)
    w = [0]*44
    ws = bytes_to_words(master_key_bytes)
    w[0:4] = ws
    for i in range(4, 44):
        temp = w[i-1]
        if i % 4 == 0:
            temp = sub_word(rot_word(temp)) ^ (RCON[i//4] << 24)
        w[i] = (w[i-4] ^ temp) & 0xFFFFFFFF
    # produce 11 round keys
    rks = []
    for r in range(11):
        rks.append(words_to_bytes(w[4*r:4*r+4]))
    return rks

# Use corrected inversion on the CPA round-10 key
r10 = bytes(last_round_key)
orig_key = invert_key_schedule_from_r10(r10)
print("Recovered master key (correct inversion):", ''.join(f'{b:02x}' for b in orig_key))

# Forward-expand and confirm we get the same round-10 key back
rk_all = expand_key_forward(orig_key)
print("Round-10 from expansion equals CPA r10? ", rk_all[10] == r10)


Recovered master key (correct inversion): 2b7e151628aed2a6abf7158809cf4f3c
Round-10 from expansion equals CPA r10?  True
